# Imports

In [5]:
import tensorflow as tf
tf.keras.backend.clear_session()

In [6]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.nn.functional as F
import torchvision
torch.use_deterministic_algorithms(True)
from tqdm import tqdm
import torch.optim as optim

import os
import time
import copy
from torchvision import datasets, models, transforms

# LeNet5 Implementation Using Tensorflow Sequential API

In [7]:
# Tensorflow Sequential Implementation
lenet_sequential_tf = tf.keras.Sequential([
    tf.keras.layers.Input(shape = (32, 32, 1)), # input
    tf.keras.layers.Conv2D(filters = 6, kernel_size = 5, activation = "relu"), #C1
    tf.keras.layers.AveragePooling2D(2), #S2
    tf.keras.layers.Conv2D(filters = 16, kernel_size = 5, activation = "relu"), #C3
    tf.keras.layers.AveragePooling2D(2), #S4,
    tf.keras.layers.Conv2D(filters = 120, kernel_size = 5, activation = "relu"), #C5
    tf.keras.layers.Dense(84, activation = "tanh"), #F6
    tf.keras.layers.Dense(10, activation = "softmax") #output
])

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy()
lenet_sequential_tf.compile(optimizer = optimizer, loss = loss, metrics = ["accuracy"])

In [8]:
lenet_sequential_tf.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 6)      │           156 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 14, 14, 6)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 10, 10, 16)     │         2,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_1             │ (None, 5, 5, 16)       │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 1, 1, 120)      │        48,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1, 1, 84)       │        10,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1, 1, 10)       │           850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 61,706 (241.04 KB)

 Trainable params: 61,706 (241.04 KB)

 Non-trainable params: 0 (0.00 B)

# LeNet5 Implementation Using Tensorflow Functional API

In [10]:
# Tensorflow functional Implementation
input_ = tf.keras.layers.Input(shape = (32, 32, 1))
c1 = tf.keras.layers.Conv2D(filters = 6, kernel_size = 5, activation = "relu")(input_)
s2 =  tf.keras.layers.AveragePooling2D(2)(c1)
c3 = tf.keras.layers.Conv2D(filters = 16, kernel_size = 5, activation = "relu")(s2)
s4 = tf.keras.layers.AveragePooling2D(2)(c3)
c5 = tf.keras.layers.Conv2D(filters = 120, kernel_size = 5, activation = "relu")(s4)
f6 = tf.keras.layers.Dense(84, activation = "tanh")(c5)
output = tf.keras.layers.Dense(10, activation = "softmax")(f6)

lenet_functional_tf = tf.keras.Model(inputs = [input_], outputs = [output])
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy()
lenet_functional_tf.compile(optimizer = optimizer, loss = loss, metrics = ["accuracy"])

In [11]:
lenet_functional_tf.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 32, 32, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 28, 28, 6)      │           156 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_2             │ (None, 14, 14, 6)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 10, 10, 16)     │         2,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_3             │ (None, 5, 5, 16)       │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 1, 1, 120)      │        48,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1, 1, 84)       │        10,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1, 1, 10)       │           850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 61,706 (241.04 KB)

 Trainable params: 61,706 (241.04 KB)

 Non-trainable params: 0 (0.00 B)

# LeNet5 Implementation Using Torch Sequential API

In [12]:
# Torch Sequential Implementation
lenet_sequential_torch = nn.Sequential(
    nn.Conv2d(1, 6, 5, 1),
    nn.ReLU(),
    nn.AvgPool2d(2),
    nn.Conv2d(6, 16, 5, 1),
    nn.ReLU(),
    nn.AvgPool2d(2),
    nn.Conv2d(16, 120, 5, 1),
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(120, 84),
    nn.Tanh(),
    nn.Linear(84, 10),
    nn.LogSoftmax(dim = 1)
)

print(lenet_sequential_torch)

Sequential(
  (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU()
  (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (4): ReLU()
  (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (6): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (7): ReLU()
  (8): Flatten(start_dim=1, end_dim=-1)
  (9): Linear(in_features=120, out_features=84, bias=True)
  (10): Tanh()
  (11): Linear(in_features=84, out_features=10, bias=True)
  (12): LogSoftmax(dim=1)
)


# LeNet5 Implementation Using Tensorflow SubClassing API

In [13]:
# Torch Sub Classing Implementation
class Lenet_SubClassing_torch(nn.Module):
  def __init__(self):
    super(Lenet_SubClassing_torch, self).__init__()
    self.cn1 = nn.Conv2d(1, 6, 5, 1)
    self.cn2 = nn.Conv2d(6, 16, 5, 1)
    self.cn3 = nn.Conv2d(16, 120, 5, 1)
    self.flatten = nn.Flatten()
    self.fc1 = nn.Linear(120, 84)
    self.fc2 = nn.Linear(84, 10)


  def forward(self, x):
    x = self.cn1(x)
    x = F.relu(x)
    x = F.avg_pool2d(x, 2)
    x = self.cn2(x)
    x = F.relu(x)
    x = F.avg_pool2d(x, 2)
    x = self.cn3(x)
    x = F.relu(x)
    x = self.flatten(x)
    x = self.fc1(x)
    x = F.tanh(x)
    x = self.fc2(x)

    out = F.log_softmax(x, dim = 1)
    return out

lenet_subclassing_torch = Lenet_SubClassing_torch()

In [14]:
model = Lenet_SubClassing_torch()
x = torch.randn(64, 1, 32, 32)
print(model(x).shape)

torch.Size([64, 10])


In [15]:
model = lenet_sequential_torch
x = torch.randn(64, 1, 32, 32)
print(model(x).shape)

torch.Size([64, 10])


In [16]:
print(model)

Sequential(
  (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (1): ReLU()
  (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (4): ReLU()
  (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (6): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (7): ReLU()
  (8): Flatten(start_dim=1, end_dim=-1)
  (9): Linear(in_features=120, out_features=84, bias=True)
  (10): Tanh()
  (11): Linear(in_features=84, out_features=10, bias=True)
  (12): LogSoftmax(dim=1)
)
